In [1]:
# Import libraries

import requests
from bs4 import BeautifulSoup
import re

In [4]:
def search_zambian_pages(limit=10):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": "intitle:Zambia",
        "format": "json",
        "srlimit": limit
    }
    response = requests.get(url, params=params).json()
    print([result["title"] for result in response["query"]["search"]])

search_zambian_pages(limit=10)

['Zambia', 'Outline of Zambia', 'President of Zambia', 'Economy of Zambia', 'China–Zambia relations', 'Geography of Zambia', 'Languages of Zambia', 'Foreign relations of Zambia', 'India–Zambia relations', 'Zambia Railways']
